**DEEP LEARNING**

Transfer Learning ResNet 50 -Rice image Dataset

GABRIEL JOSHUA . R

### Import necessary libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense

Assuming you have a dummy dataset stored in '/content/Rice_Image_Dataset'

Make sure to organize the data into 'train', 'val', and 'test' folders


### Data augmentation and loading the dataset

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

### Splitting the data into train, validation, and test sets
### Here, '/content/Rice_Image_Dataset' should be the path to your dataset folder


In [ ]:
train_data = train_datagen.flow_from_directory(
    '/content/Rice_Image_Dataset/train',
    target_size=(256, 256),
    batch_size=32
)

val_data = val_datagen.flow_from_directory(
    '/content/Rice_Image_Dataset/val',
    target_size=(256, 256),
    batch_size=32
)

test_data = test_datagen.flow_from_directory(
    '/content/Rice_Image_Dataset/test',
    target_size=(256, 256),
    batch_size=32
)

num_classes = train_data.num_classes
nb_train_samples = train_data.samples
nb_val_samples = val_data.samples

### Function to create a custom ResNet50 model with fine-tuning options

In [ ]:
def Resnet50_model_finetuning(freeze_baselayers):
    base_model = tf.keras.applications.resnet50.ResNet50(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(256, 256, 3),
        classes=5  # Change this to the number of classes in your dataset
    )
    
    # Freeze base model layers if specified
    if freeze_baselayers:
        base_model.trainable = False
        
    # Building the new classification layers on top of the base model
    input = base_model.inputs
    mod = base_model.output
    mod = tf.keras.layers.Flatten()(mod)
    mod = Dense(64, activation="relu")(mod)
    mod = Dense(64, activation="relu")(mod)
    output = Dense(num_classes, activation='softmax')(mod)

    # Creating the new model
    new_model = tf.keras.Model(inputs=input, outputs=output)
    return new_model

### Creating the model with fine-tuned ResNet50 base and custom classification layers


In [ ]:
model = Resnet50_model_finetuning(freeze_baselayers=True)

### Compiling the model with optimizer, loss, and metrics

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

### Training the model

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=val_data)

### Plotting the training and validation accuracy

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend(["train", "test"], loc="upper left")
plt.show()

### Plotting the training and validation loss

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("loss")
plt.xlabel("epochs")
plt.legend(["train", "test"], loc="upper left")
plt.show()

### Evaluating the model on the test set

In [ ]:
results = model.evaluate(test_data, verbose=0)
print("Loss: {:.5f}".format(results[0]))
print("Accuracy: {:.2f}%".format(results[1] * 100))